# Day 11
https://adventofcode.com/2020/day/11

In [1]:
import aocd
data = aocd.get_data(year=2020, day=11)

In [2]:
from dataclasses import dataclass
from typing import Set

In [3]:
@dataclass(frozen=True)
class Point():
    y: int
    x: int
    
    def __add__(self, other):
        return Point(self.y+other.y, self.x+other.x)

In [4]:
@dataclass(frozen=True)
class Seat():
    location: Point
    occupied: bool
    neighbours: Set[Point]
    
    @classmethod
    def all_from_input(cls, text, neighbour_policy='adjacent'):
        lines = text.split('\n')
        
        points = {}
        for y, line in enumerate(lines):
            for x, char in enumerate(line):
                if char != '.':
                    points[Point(y, x)] = (char == '#')
        
        neighbours = {}
        
        for point in points.keys():
            max_x = max(point.x for point in points.keys())
            max_y = max(point.y for point in points.keys())
            neighbours[point] = set()
            for compass in (
                Point(-1, -1), Point(-1, 0), Point(-1, 1),
                Point(0, -1), Point(0, 1),
                Point(1, -1), Point(1, 0), Point(1, 1),
            ):
                nearby = point + compass
                if neighbour_policy == 'nearest':
                    while (nearby not in points and nearby.x >= 0 and nearby.y >= 0
                           and nearby.x <= max_x and nearby.y <= max_y):
                        nearby += compass
                if nearby in points:
                    neighbours[point].add(nearby)
        
        return dict((point, cls(point, occupied, neighbours.get(point, set())))
                    for point, occupied in points.items())
    
    def occupied_neighbours(self, seats):
        return sum(1 for neighbour in self.neighbours if seats[neighbour].occupied)
    
    def progress(self, seats, occupied_tolerance=4):
        if (not self.occupied) and self.occupied_neighbours(seats) == 0:
            return Seat(self.location, True, self.neighbours)
        if self.occupied and self.occupied_neighbours(seats) >= occupied_tolerance:
            return Seat(self.location, False, self.neighbours)
        
        return Seat(self.location, self.occupied, self.neighbours)

In [5]:
def progress_until_stable(seats, occupied_tolerance=4):
    oldseats = ""
    while oldseats != repr(seats):
        oldseats = repr(seats)
        seats = dict((point, seat.progress(seats, occupied_tolerance)) for point, seat in seats.items())
    return seats

In [6]:
seats = Seat.all_from_input(data)
stable = progress_until_stable(seats)
p1 = sum(1 for seat in stable.values() if seat.occupied)
print('Part 1: {}'.format(p1))

Part 1: 2275


In [7]:
seats = Seat.all_from_input(data, 'nearest')
stable = progress_until_stable(seats, occupied_tolerance=5)
p2 = sum(1 for seat in stable.values() if seat.occupied)
print('Part 2: {}'.format(p2))

Part 2: 2121
